In [1]:
from cerebro.backend import DaskBackend

from cerebro.storage import LocalStore, HDFSStore

# Model selection/AutoML methods.
from cerebro.tune import GridSearch, RandomSearch, TPESearch

# Utility functions for specifying the search space.
from cerebro.tune import hp_choice, hp_uniform, hp_quniform, hp_loguniform, hp_qloguniform

import tensorflow as tf
import dask.dataframe as dd
import pandas as pd
import os

2021-11-13 18:22:22.367714: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-11-13 18:22:22.367736: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Define dictionary containing the parameter search space.
search_space = {
    'lr': hp_choice([0.01, 0.001]),
    'batch_size': hp_choice([2, 4]),
}

def get_basic_model(params):
    print("calling get basic model")
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_dim=10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
    opt = tf.keras.optimizers.Adam(lr=params['lr'])
    model.compile(optimizer=opt,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model


store = LocalStore(prefix_path='/users/pratadiy/cloudlab-scripts-cerebro/expts')
checkpoint_paths = '/users/pratadiy/cloudlab-scripts-cerebro/expts'
backend = DaskBackend(num_workers=4, estimator_gen_fn=get_basic_model, checkpoint_base_path=checkpoint_paths)

Client dashboard:  http://127.0.0.1:8787/status
CEREBRO-Dask => Time: 2021-11-13 18:22:25, Running 4 Workers


In [3]:
csv_file = tf.keras.utils.get_file('/users/pratadiy/heart.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/heart.csv')
df = pd.read_csv(csv_file)

In [4]:
pq_file = '/users/pratadiy/heart.parquet'
df.to_parquet(pq_file)
df = dd.read_parquet(pq_file)

In [5]:
model_selection = GridSearch(backend=backend, store=store, estimator_gen_fn=None, search_space=search_space,
            num_epochs=2, evaluation_metric='loss',
            feature_columns=[list(df.columns)[:-1]], label_columns=[list(df.columns)[-1]])

In [6]:
model = model_selection.fit(df)

CEREBRO => Time: 2021-11-13 18:22:25, Initializing Workers
Data workers initialized
CEREBRO => Time: 2021-11-13 18:22:25, Preparing Data
{0: 'tcp://127.0.0.1:33653', 1: 'tcp://127.0.0.1:39727', 2: 'tcp://127.0.0.1:42067', 3: 'tcp://127.0.0.1:43971'}
Data preparation done
CEREBRO => Time: 2021-11-13 18:22:25, Initializing Data Loaders
Data loaders initialized
CEREBRO => Time: 2021-11-13 18:22:25, Launching Model Selection Workload
I ran
Model config length:  4
[{'lr': 0.01, 'batch_size': 2}, {'lr': 0.01, 'batch_size': 4}, {'lr': 0.001, 'batch_size': 2}, {'lr': 0.001, 'batch_size': 4}]
Workers are initialized
running model:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_0/cp.ckpt on worker:0
model assigned:0 on worker:0 status:pending
running model:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_1/cp.ckpt on worker:1
model assigned:1 on worker:1 status:pending
running model:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_2/cp.ckpt on worker:2
model assigned:2 on worker:2 stat

2021-11-13 18:22:26.377457: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-11-13 18:22:26.377479: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-11-13 18:22:26.452991: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-11-13 18:22:26.453013: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-11-13 18:22:26.478618: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or dire

calling train_model for model:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_0/cp.ckpt
calling get basic model
Model name:  0  Worker:  0  Config:  {'name': 'Adam', 'learning_rate': 0.01, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}  Batch size:  2
36/36 [==============================] - 0s 450us/step - loss: 0.6931 - accuracy: 0.7222
calling train_model for model:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_2/cp.ckpt
calling get basic model
Model name:  2  Worker:  2  Config:  {'name': 'Adam', 'learning_rate': 0.001, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}  Batch size:  2
30/30 [==============================] - 0s 421us/step - loss: 1.2133 - accuracy: 0.1000
calling train_model for model:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_1/cp.ckpt
calling get basic model
Model name:  1  Worker:  1  Config:  {'name': 'Adam', 'learning_rate': 0.01, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 

done model:3 on worker:0
m:3 val:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_3/cp.ckpt
done model:0 on worker:1
m:0 val:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_0/cp.ckpt
done model:1 on worker:2
m:1 val:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_1/cp.ckpt
done model:2 on worker:3
m:2 val:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_2/cp.ckpt
Implemented train_for_one_epoch
I ran
Model config length:  4
[{'lr': 0.01, 'batch_size': 2}, {'lr': 0.01, 'batch_size': 4}, {'lr': 0.001, 'batch_size': 2}, {'lr': 0.001, 'batch_size': 4}]
Workers are initialized
running model:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_0/cp.ckpt on worker:0
model assigned:0 on worker:0 status:pending
running model:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_1/cp.ckpt on worker:1
model assigned:1 on worker:1 status:pending
running model:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_2/cp.ckpt on worker:2
model assigned:2 on worker:2 status:pending
running mod

calling train_model for model:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_1/cp.ckpt
calling get basic model
Model name:  1  Worker:  0  Config:  {'name': 'Adam', 'learning_rate': 0.01, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}  Batch size:  4
 1/18 [>.............................] - ETA: 0s - loss: 0.6932 - accuracy: 0.7500calling train_model for model:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_2/cp.ckpt
calling get basic model
Model name:  2  Worker:  1  Config:  {'name': 'Adam', 'learning_rate': 0.001, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}  Batch size:  2
18/18 [==============================] - 0s 445us/step - loss: 0.6931 - accuracy: 0.7222
calling train_model for model:/users/pratadiy/cloudlab-scripts-cerebro/exptsmodel_3/cp.ckpt
calling get basic model
Model name:  3  Worker:  2  Config:  {'name': 'Adam', 'learning_rate': 0.001, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsil